# SQL Case Study Part 2

## The assignment

- Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!
- Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order
- Q12: Find the facilities with their usage by member, but not guests
- Q13: Find the facilities usage by month, but not guests

## Setting up the server

In [20]:
from sqlalchemy import create_engine, text, inspect
import pandas as pd

engine = create_engine('sqlite:///country_club.db')
inspect(engine).get_table_names()


['Bookings', 'Facilities', 'Members']

## Q10

Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [22]:
query10 = \
    """SELECT name, SUM(revenue) as total_revenue
        FROM (
            SELECT name,
            (
                CASE WHEN memid = 0 THEN slots * guestcost
                ELSE slots * membercost
                END
            ) AS revenue
            FROM Members 
            INNER JOIN Bookings USING (memid)
            INNER JOIN Facilities USING (facid)
        )
        GROUP BY name 
        HAVING SUM(revenue) < 1000 
        ORDER BY total_revenue;"""

def get_results_frame(query, engine):
    with engine.connect() as conn:
        rs = conn.execute(text(query))
        results = pd.DataFrame(rs.fetchall(), columns=rs.keys())
    return results

results10 = get_results_frame(query10, engine)
results10

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


## Q11

Produce a report of members and who recommended them in alphabetic surname,firstname order

In [31]:
query11 = """SELECT m.surname, m.firstname, r.firstname || ' ' || r.surname as recommender
    FROM Members as m
    LEFT JOIN Members as r
    ON m.recommendedby = r.memid
    WHERE m.memid <> 0
    ORDER BY m.surname, m.firstname;
"""

results11 = get_results_frame(query11, engine)
results11

,surname,firstname,recommender
0,Bader,Florence,Ponder Stibbons
1,Baker,Anne,Ponder Stibbons
2,Baker,Timothy,Jemima Farrell
3,Boothe,Tim,Tim Rownam
4,Butters,Gerald,Darren Smith
5,Coplin,Joan,Timothy Baker
6,Crumpet,Erica,Tracy Smith
7,Dare,Nancy,Janice Joplette
8,Farrell,David,None
9,Farrell,Jemima,None


## Q12

Find the facilities with their usage by member, but not guests

In [36]:
# interpreting this as total slots booked by members
# recalling taht Guest memid is 0
query12 = """SELECT name, SUM(slots) as total_slots_booked
    FROM Facilities
    INNER JOIN Bookings USING (facid)
    WHERE memid <> 0
    GROUP BY facid
    ORDER BY name;
"""

results12 = get_results_frame(query12, engine)
results12

,name,total_slots_booked
0,Badminton Court,1086
1,Massage Room 1,884
2,Massage Room 2,54
3,Pool Table,856
4,Snooker Table,860
5,Squash Court,418
6,Table Tennis,794
7,Tennis Court 1,957
8,Tennis Court 2,882


## Q13

Find the facilities usage by month, but not guests

In [37]:
# again, interpreting usage as slots booked
# assuming also that this is supposed to be by facility, as above

query13 = """SELECT name, month, SUM(slots) AS total_slots_booked
    FROM Facilities
    INNER JOIN (
        SELECT facid, memid, slots, (
            CASE WHEN starttime LIKE '2012-07%' THEN 7
            WHEN starttime LIKE '2012-08%' THEN 8
            ELSE 9
            END
        ) AS month
        FROM Bookings
    ) AS bookings_months USING (facid)
    WHERE memid <> 0
    GROUP BY facid, month
    ORDER BY name, month;
"""

results13 = get_results_frame(query13, engine)
results13

,name,month,total_slots_booked
0,Badminton Court,7,165
1,Badminton Court,8,414
2,Badminton Court,9,507
3,Massage Room 1,7,166
4,Massage Room 1,8,316
5,Massage Room 1,9,402
6,Massage Room 2,7,8
7,Massage Room 2,8,18
8,Massage Room 2,9,28
9,Pool Table,7,110
